In [1]:
import pandas as pd
import numpy as np
import mlflow
from prophet import Prophet
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import logging
import os
from apps.ml.features import SensorFeatureTransformer

logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)

tracking_uri = "http://mlflow:5000" if os.getenv("DOCKER_ENV") == "true" else "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("Forecasting Models")

print(f"MLflow tracking URI set to: {mlflow.get_tracking_uri()}")

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
mkdir -p failed for path /.config/matplotlib: [Errno 13] Permission denied: '/.config'


Matplotlib created a temporary cache directory at /tmp/matplotlib-t12id4nk because there was an issue with the default path (/.config/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


MLflow tracking URI set to: http://mlflow:5000


In [2]:
# --- Data Preparation (Consistent for both models) ---
df = pd.read_csv('data/sensor_data.csv', parse_dates=['timestamp'])
df['timestamp'] = df['timestamp'].dt.tz_localize(None)

sensor_id = 'sensor-001'
df_sensor = df[df['sensor_id'] == sensor_id].copy()

# 80/20 Train/Test Split
split_point = int(len(df_sensor) * 0.8)
train_df_raw = df_sensor.iloc[:split_point]
test_df_raw = df_sensor.iloc[split_point:]

y_true = test_df_raw['value'].values

print(f"Data prepared for sensor: {sensor_id}")

Data prepared for sensor: sensor-001


In [3]:
# --- 1. Prophet Hyperparameter Tuning ---
print("\n--- Starting Prophet Tuning ---")

# Define a simple grid of hyperparameters to test
param_grid = {
    'changepoint_prior_scale': [0.01, 0.05, 0.1],
    'seasonality_prior_scale': [1.0, 5.0, 10.0]
}

best_mae = float('inf')
best_params = {}

# Prepare data for Prophet
train_df_prophet = train_df_raw[['timestamp', 'value']].rename(columns={'timestamp': 'ds', 'value': 'y'})

for cps in param_grid['changepoint_prior_scale']:
    for sps in param_grid['seasonality_prior_scale']:
        with mlflow.start_run(run_name=f"Prophet_Tuning_cps_{cps}_sps_{sps}", nested=True):
            params = {'changepoint_prior_scale': cps, 'seasonality_prior_scale': sps}
            mlflow.log_params(params)
            mlflow.log_param("model_type", "Prophet_Tuned")

            model = Prophet(**params, daily_seasonality=True, weekly_seasonality=False, yearly_seasonality=False)
            model.fit(train_df_prophet)
            
            future_df = model.make_future_dataframe(periods=len(test_df_raw), freq='5min')
            forecast_df = model.predict(future_df)
            y_pred = forecast_df['yhat'][-len(test_df_raw):].values
            
            mae = mean_absolute_error(y_true, y_pred)
            mlflow.log_metric("mae", mae)
            
            if mae < best_mae:
                best_mae = mae
                best_params = params
                # Log the best model so far
                mlflow.prophet.log_model(model, "best_prophet_model")
                mlflow.set_tag("status", "best_candidate")

print(f"Best Prophet MAE: {best_mae:.4f} with params: {best_params}")

2025/08/22 12:50:40 WARNING mlflow.tracking.context.registry: Encountered unexpected error during resolving tags: 'getpwuid(): uid not found: 1000'



--- Starting Prophet Tuning ---


12:50:40 - cmdstanpy - INFO - Chain [1] start processing


12:50:40 - cmdstanpy - INFO - Chain [1] done processing


2025/08/22 12:50:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

2025/08/22 12:50:44 WARNING mlflow.tracking.context.registry: Encountered unexpected error during resolving tags: 'getpwuid(): uid not found: 1000'


🏃 View run Prophet_Tuning_cps_0.1_sps_5.0 at: http://mlflow:5000/#/experiments/5/runs/37f93c492f6e4db1bc373c5561173017
🧪 View experiment at: http://mlflow:5000/#/experiments/5


12:50:44 - cmdstanpy - INFO - Chain [1] start processing


12:50:44 - cmdstanpy - INFO - Chain [1] done processing


🏃 View run Prophet_Tuning_cps_0.1_sps_10.0 at: http://mlflow:5000/#/experiments/5/runs/01e5622c29a54f079e02a0e3a6319b22
🧪 View experiment at: http://mlflow:5000/#/experiments/5
Best Prophet MAE: 2.8258 with params: {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 5.0}


In [4]:
# --- 2. Challenger Model: LightGBM with Lag Features ---
print("\n--- Starting LightGBM Challenger Model ---")

with mlflow.start_run(run_name="LightGBM_Challenger_v1"):
    mlflow.log_param("model_type", "LightGBM")
    
    # --- Feature Engineering ---
    # Create lag features for forecasting the next value
    feature_transformer = SensorFeatureTransformer(n_lags=12, scale_columns=['value'])
    
    # Transform train data
    X_train_transformed = feature_transformer.fit_transform(train_df_raw)
    # Create target: next value (shift -1 means we predict the next value)
    y_train_full = train_df_raw['value'].shift(-1)
    
    # Remove the last row since it doesn't have a target (NaN after shift)
    X_train_clean = X_train_transformed.iloc[:-1]
    y_train_clean = y_train_full.iloc[:-1]
    
    # Transform test data
    X_test_transformed = feature_transformer.transform(test_df_raw)
    y_test_full = test_df_raw['value'].shift(-1)
    
    # Remove the last row since it doesn't have a target
    X_test_clean = X_test_transformed.iloc[:-1]
    y_test_clean = y_test_full.iloc[:-1]
    
    # Check for any remaining NaN values and handle them
    print(f"NaN values in X_train: {X_train_clean.isna().sum().sum()}")
    print(f"NaN values in y_train: {y_train_clean.isna().sum()}")
    print(f"NaN values in X_test: {X_test_clean.isna().sum().sum()}")
    print(f"NaN values in y_test: {y_test_clean.isna().sum()}")
    
    # Drop any remaining NaN rows
    if X_train_clean.isna().any().any() or y_train_clean.isna().any():
        valid_train_idx = ~(X_train_clean.isna().any(axis=1) | y_train_clean.isna())
        X_train_clean = X_train_clean[valid_train_idx]
        y_train_clean = y_train_clean[valid_train_idx]
        print(f"Dropped {(~valid_train_idx).sum()} training rows with NaN")
    
    if X_test_clean.isna().any().any() or y_test_clean.isna().any():
        valid_test_idx = ~(X_test_clean.isna().any(axis=1) | y_test_clean.isna())
        X_test_clean = X_test_clean[valid_test_idx]
        y_test_clean = y_test_clean[valid_test_idx]
        print(f"Dropped {(~valid_test_idx).sum()} test rows with NaN")
    
    print(f"Final training set size: {len(X_train_clean)}")
    print(f"Final test set size: {len(X_test_clean)}")
    
    # --- Model Training ---
    lgbm = lgb.LGBMRegressor(random_state=42, verbosity=-1)
    mlflow.log_params(lgbm.get_params())
    lgbm.fit(X_train_clean, y_train_clean)
    
    # --- Evaluation ---
    y_pred_lgbm = lgbm.predict(X_test_clean)
    mae_lgbm = mean_absolute_error(y_test_clean, y_pred_lgbm)
    mlflow.log_metric("mae", mae_lgbm)
    
    print(f"LightGBM MAE: {mae_lgbm:.4f}")

    # --- Log Model ---
    mlflow.lightgbm.log_model(lgbm, "model", registered_model_name="lightgbm_forecaster_challenger")

print("\n--- Experiment session complete! ---")

2025/08/22 12:50:44 WARNING mlflow.tracking.context.registry: Encountered unexpected error during resolving tags: 'getpwuid(): uid not found: 1000'



--- Starting LightGBM Challenger Model ---
NaN values in X_train: 0
NaN values in y_train: 0
NaN values in X_test: 0
NaN values in y_test: 0
Final training set size: 479
Final test set size: 119


/app/apps/ml/features.py:75: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.ffill().bfill())
/app/apps/ml/features.py:75: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.ffill().bfill())


LightGBM MAE: 3.0994


2025/08/22 12:50:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Successfully registered model 'lightgbm_forecaster_challenger'.


2025/08/22 12:50:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lightgbm_forecaster_challenger, version 1


🏃 View run LightGBM_Challenger_v1 at: http://mlflow:5000/#/experiments/5/runs/415b118116a34132b3f390831762dd1e
🧪 View experiment at: http://mlflow:5000/#/experiments/5

--- Experiment session complete! ---


Created version '1' of model 'lightgbm_forecaster_challenger'.
